In [2]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install pyteomics
!{sys.executable} -m pip install pandas


  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=883c19b9a95d14fa502ed1f41b70441b5c85e47d56e23cae944076d0698b1fe7
  Stored in directory: c:\users\kyle\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built termcolor
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


In [11]:
# happyWhale - Data Walk

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import glob
from tqdm import tqdm
import random
import os
from sklearn.model_selection import train_test_split


# Input data files are available in the read-only "../input/" directory on kaggle
#  - to access locally lets add the same one to this environment
#  - to work with this add a folder structure of ./kaggle/input/
#  - unzip the train.csv in the input folder - it is only about 3 mb

# Mapping the Directories
train_path = '/kaggle/train.csv'

# KRP: Get the training data set
train_csv = './kaggle/train.csv'
whale_df = pd.read_csv(train_csv)

# KRP: Adding the edits to fix known issues in the dataset suggested by Aleksey Alekseev in the challenge discussions
#   see: https://www.kaggle.com/c/happy-whale-and-dolphin/discussion/305574
whale_df.species.replace({"globis": "short_finned_pilot_whale",
                          "pilot_whale": "short_finned_pilot_whale",
                          "kiler_whale": "killer_whale",
                          "bottlenose_dolpin": "bottlenose_dolphin"}, inplace=True)

# KRP: Get number of unique species
species = whale_df.species.unique()
print("The Species represented in this set:")
print(species)

outputSize = len(species) #Number of unique species to identify
print("This gives us a total of " + str(outputSize) + " species")
print("=========================================================================")


# KRP: Get number of individual whales
individuals = whale_df.individual_id.unique()
print("The list of individual whales: ")
print(individuals)

print("This gives us a total of " + str(len(individuals)) + " individuals")
print("=========================================================================")

# KRP: 15000 individuals is a lot, lets take a look at what individuals are showing up a lot
# Some additional characterization

sorted_df = whale_df
sorted_df['count'] = sorted_df.groupby('individual_id')['individual_id'].transform('count')
sorted_df.sort_values('count', inplace=True, ascending=False)
sorted_df['individual_id'].mask(sorted_df['count'] <=9 ,'unknown', inplace=True)
sorted_df['count'] = sorted_df.groupby('individual_id')['individual_id'].transform('count')
sorted_df.drop_duplicates(subset='individual_id', keep='first', inplace=True)
sorted_df = sorted_df.head(1000)
print("Sorting the list by number of individual occurrences, ignoring anything with fewer than 8 photos")
print(sorted_df)





The Species represented in this set:
['melon_headed_whale' 'humpback_whale' 'false_killer_whale'
 'bottlenose_dolphin' 'beluga' 'minke_whale' 'fin_whale' 'blue_whale'
 'gray_whale' 'southern_right_whale' 'common_dolphin' 'killer_whale'
 'short_finned_pilot_whale' 'dusky_dolphin' 'long_finned_pilot_whale'
 'sei_whale' 'spinner_dolphin' 'cuviers_beaked_whale' 'spotted_dolphin'
 'brydes_whale' 'commersons_dolphin' 'white_sided_dolphin'
 'rough_toothed_dolphin' 'pantropic_spotted_dolphin' 'pygmy_killer_whale'
 'frasiers_dolphin']
This gives us a total of 26 species
The list of individual whales: 
['cadddb1636b9' '1a71fbb72250' '60008f293a2b' ... '3509cb6a8504'
 'e880e47c06a4' 'bc6fcab946c4']
This gives us a total of 15587 individuals
Sorting the list by number of individual occurrences, ignoring anything with fewer than 8 photos
                    image             species individual_id  count
46293  e8edb1be7b61e4.jpg         minke_whale  37c7aba965a5    400
12683  3fea89c47202e3.jpg    

Deprecated Demo Code